The goal of this notebook is to demonstrate loading a grib file using pygrib and then converting it into a pandas data frame

In [2]:
#Import necessary libraries.
import pygrib
import pandas as pd
import numpy as np

#Open the GRIB file.
grib_file = '../Data/a3d700c0638cdb3a7d8b291e79517521/data.grib'
grbs = pygrib.open(grib_file)

#Iterate through GRIB messages and extract data:
data = []
for grb in grbs:
    # Extract relevant information
    latitude = grb['latitudeOfFirstGridPoint']
    longitude = grb['longitudeOfFirstGridPoint']
    values = grb['values']
    
    # Append data to list
    data.append({
        'latitude': latitude,
        'longitude': longitude,
        'values': values,
        'parameter': grb['parameterName'],
        'level': grb['level']
    })
#Create a pandas DataFrame.
df = pd.DataFrame(data)

#Handle multiple grid points per message: If each GRIB message contains a grid of data points, flatten the data and create new rows for each point.
all_data = []
for _, row in df.iterrows():
    lat_start = row['latitude']
    lon_start = row['longitude']
    values = row['values']
    ni = grb['Ni']  # Number of grid points in i direction
    nj = grb['Nj']  # Number of grid points in j direction
    lat_step = grb['jDirectionIncrementInDegrees']
    lon_step = grb['iDirectionIncrementInDegrees']

    for i in range(nj):
        for j in range(ni):
            all_data.append({
                'latitude': lat_start + i * lat_step,
                'longitude': lon_start + j * lon_step,
                'value': values[i * ni + j],
                'parameter': row['parameter'],
                'level': row['level']
            })

df_expanded = pd.DataFrame(all_data)

#Print or further process the DataFrame.
print(df_expanded.head())

IndexError: index 251 is out of bounds for axis 0 with size 251